# Homework 4
In this homework, we'll deploy the ride duration model in batch mode. Like in homework 1 and 3, we'll use the FHV data.

You'll find the starter code in the [homework][homework-src] directory.

## Q1. Notebook

We'll start with the same notebook we ended up with in homework 1.

We cleaned it a little bit and kept only the scoring part. Now it's in [homework/starter.ipynb](./homework4/starter.ipynb).

Run this notebook for the February 2021 FVH data.

What's the mean predicted duration for this dataset?

* 11.19
* 16.19
* 21.19
* 26.19

[homework-src]: https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/04-deployment/homework

Answer: 16.19

## Q2. Preparing the output

Like in the course videos, we want to prepare the dataframe with the output.

First, let's create an artificial `ride_id` column:

```
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
```

Next, write the ride id and the predictions to a dataframe with results.

Save it as parquet:

```
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
```

What's the size of the output file?

* 9M
* 19M
* 29M
* 39M

Make sure you use the snippet above for saving the file. It should contain only these two columns. For this question, don't change the dtypes of the columns and use pyarrow, not fastparquet.

In [1]:
import os
 
file_size = os.path.getsize('./homework4/ride_predictions.parquet') / 1024**2
print(f'File Size is: {file_size:,.1f} MB.')

File Size is: 18.8 MB.


## Q3. Creating the scoring script

Now let's turn the notebook into a script.

Which command you need to execute for that?

In [2]:
!jupyter nbconvert --to script ./homework4/starter.ipynb

[NbConvertApp] Converting notebook ./homework4/starter.ipynb to script
[NbConvertApp] Writing 1268 bytes to homework4\starter.py


Answer: `jupyter nbconvert ...`

## Q4. Virtual environment

Now let's put everything into a virtual environment. We'll use pipenv for that.

Install all the required libraries. Pay attention to the Scikit-Learn version: check the starter notebook for details.

After installing the libraries, pipenv creates two files: `Pipfile` and `Pipfile.lock`. The `Pipfile.lock` file keeps the hashes of the dependencies we use for the virtual env.

What's the first hash for the Scikit-Learn dependency?

In [3]:
import json

with open('Pipfile.lock') as fin:
    doc = json.load(fin)
    print(doc['default']['scikit-learn']['hashes'][0])

sha256:0403ad13f283e27d43b0ad875f187ec7f5d964903d92d1ed06c51439560ecea0
